# 5. 케라스와 텐서플로우

__감사말__

프랑소와 숄레의 [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff) 3장에 사용된 코드에 대한 설명을 담고 있으며 텐서플로우 2.6 버전 이상에서 작성되었습니다. 소스코드를 공개한 저자에게 감사드립니다.

__구글 코랩 설정__

'런타임 -> 런타임 유형 변경' 메뉴에서 GPU를 지정한다.
TensorFlow 버전을 확인하려면 아래 명령문을 실행한다.

In [1]:
import tensorflow as tf
tf.__version__

'2.17.0'

TensorFlow가 GPU를 사용하는지 여부를 확인하려면 아래 명령문을 실행한다.
아래와 같은 결과가 나오면 GPU가 제대로 지원됨을 의미한다.

```
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
```

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## 케라스 핵심 API

### 층 API

**예제: `Dense` 층 상세**

`Dense` 클래스와 유사하게 작동하는 클래스를 직접 정의하려면
상속해야 하는 `keras.layers.Layer` 클래스의 `__call()__` 메서드에 의해 호출되는
`build()` 메서드와 `call()` 메서드를 구현해야 한다.
아래 `SimpleDense` 클래스가 `Dense` 클래스의 기능을 단순화하여 구현한다.

In [3]:
from tensorflow import keras

class SimpleDense(keras.layers.Layer):

    def __init__(self, units, activation=None):
        super().__init__()
        self.units = units
        self.activation = activation

    def build(self, input_shape):
        input_dim = input_shape[-1]   # 입력 샘플의 특성 수
        self.W = self.add_weight(shape=(input_dim, self.units),
                                 initializer="random_normal")
        self.b = self.add_weight(shape=(self.units,),
                                 initializer="zeros")

    def call(self, inputs):
        y = tf.matmul(inputs, self.W) + self.b
        if self.activation is not None:
            y = self.activation(y)
        return y

### 모델 API

`Sequential` 모델처럼 작동하는 클래스를 직접 정의한다.

In [4]:
class MySequential(keras.Model):
    def __init__(self, list_layers): # 층들의 리스트 지정
        super().__init__()
        self.list_layers = list_layers

    # 포워드 패스: 층과 층을 연결하는 방식으로 구현
    def call(self, inputs):
        outputs = inputs
        for layer in self.list_layers:
            outputs = layer(outputs)
        return outputs

아래 두 개의 층을 이용하여 모델을 지정하고 다중 클래스 분류 모델에 맞게 모델을 컴파일한다.

In [5]:
layer_1 = SimpleDense(units=512, activation=tf.nn.relu)   # 첫째 밀집층
layer_2 = SimpleDense(units=10, activation=tf.nn.softmax) # 둘째 밀집층

In [6]:
model = MySequential([layer_1, layer_2])

### 모델 컴파일 API

모델의 훈련을 위해서 먼저 다음 세 가지 설정을 추가로 지정해야 한다.

- 손실 함수: 훈련 중 모델의 성능이 얼마나 나쁜지 측정.
    미분가능한 함수이어야 하며 옵티마이저가 역전파를 통해
    모델의 성능을 향상시키는 방향으로 모델의 가중치를 업데이트할 때
    참고하는 함수임.
- 옵티마이저: 백워드 패스와 역전파를 담당하는 알고리즘
- 평가지표: 훈련과 테스트 과정을 모니터링 할 때 사용되는 모델 평가 지표.
    옵티마이저와 손실함수와는 달리 훈련에 관여하지 않으면서
    모델 성능 평가에 사용됨.

In [7]:
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

### 모델 훈련 API

모델을 컴파일한 다음에 `fit()` 메서드를 호출하면
모델은 스텝과 에포크 단위로 반복되는 **훈련 루프**<font size='2'>training loop</font>를
지정된 횟수만큼 또는 학습이 충분히 이루어졌다는 평가가 내려질 때까지
반복하는 훈련을 시작한다.

In [17]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [18]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255   # 0과 1사이의 값
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255     # 0과 1사이의 값

**지도 학습 모델 훈련**

In [19]:
training_history = model.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9919 - loss: 0.0285
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9937 - loss: 0.0227
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9952 - loss: 0.0166
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9974 - loss: 0.0115
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9977 - loss: 0.0094


**`History` 객체: 훈련 결과**

훈련이 종료되면 `fit()` 메서드는 `History` 객체를 반환하며,
`history` 속성에 훈련 과정 중에 측정된 손실값, 평가지표를 에포크 단위로 기억한다.

In [11]:
training_history.history

{'accuracy': [0.9199833273887634,
  0.9668833613395691,
  0.9786999821662903,
  0.9837833046913147,
  0.9883166551589966],
 'loss': [0.2791374921798706,
  0.11251720041036606,
  0.07248896360397339,
  0.05324174463748932,
  0.03904753550887108]}

**검증셋 활용**

아래 과정은 훈련셋와 검증셋를 수동으로 구분하는 방법을 보여준다.

- `np.random.permutation()` 함수는 숫자들을 무작위로 섞는다.
    이를 이용하여 훈련세트의 인덱스를 무작위로 섞는다.

In [20]:
import numpy as np

indices_permutation = np.random.permutation(len(train_images))

- 무작위로 섞인 인덱스를 이용하여 데이터셋으르 재정렬 한다.

In [21]:
shuffled_inputs = train_images[indices_permutation]
shuffled_targets = train_labels[indices_permutation]

- 재정렬된 데이터셋의 30%를 검증셋로 분류한다.

In [22]:
num_validation_samples = int(0.3 * len(train_images))

val_inputs = shuffled_inputs[:num_validation_samples]
val_targets = shuffled_targets[:num_validation_samples]

- 나머지는 훈련셋로 지정한다.

In [23]:
training_inputs = shuffled_inputs[num_validation_samples:]
training_targets = shuffled_targets[num_validation_samples:]

*훈련 중 모델 검증*

- 훈련셋를 대상으로 하는 훈련과 검증셋를 대상으로 하는 평가를 동시에 진행할 수 있다.

In [24]:
training_history = model.fit(
    training_inputs,
    training_targets,
    epochs=5,
    batch_size=128,
    validation_data=(val_inputs, val_targets)
)

Epoch 1/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9991 - loss: 0.0055 - val_accuracy: 0.9985 - val_loss: 0.0075
Epoch 2/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9994 - loss: 0.0039 - val_accuracy: 0.9987 - val_loss: 0.0065
Epoch 3/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9997 - loss: 0.0028 - val_accuracy: 0.9970 - val_loss: 0.0107
Epoch 4/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9997 - loss: 0.0019 - val_accuracy: 0.9971 - val_loss: 0.0093
Epoch 5/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 0.9979 - val_loss: 0.0075


`History` 객체는 훈련셋 뿐만 아니라 검증셋를 대상으로도 손실값과 평가지표를 기억한다.

In [25]:
training_history.history

{'accuracy': [0.9985476136207581,
  0.9991666674613953,
  0.9995714426040649,
  0.9998095035552979,
  0.9999285936355591],
 'loss': [0.006874954793602228,
  0.004547948017716408,
  0.003098522312939167,
  0.001894774497486651,
  0.0012853598454967141],
 'val_accuracy': [0.9984999895095825,
  0.9987221956253052,
  0.996999979019165,
  0.9971110820770264,
  0.9978888630867004],
 'val_loss': [0.007506505120545626,
  0.006526042707264423,
  0.010659652762115002,
  0.009290579706430435,
  0.007476788945496082]}

### 학습된 모델 평가 API

훈련 후에 테스트셋을 이용하여 평가하려면 `evaluate()` 메서드를 이용한다.

In [26]:
loss_and_metrics = model.evaluate(test_images, test_labels, batch_size=128)

79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9800 - loss: 0.0836


In [27]:
print(loss_and_metrics)

[0.07060831040143967, 0.9821000099182129]


### 실전 모델 활용 API

실전에 배치된 모델은 새로이 입력된 데이터에 대한 예측을 실행한다.
학습된 모델의 예측값은 `predict()` 메서드를 활용하여 계산한다.
예측 또한 지정 크기의 배치 단위로 실행된다.

In [28]:
predictions = model.predict(test_images, batch_size=128)
print(predictions[:5])

79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
[[9.3658130e-11 1.5461395e-13 3.3464354e-10 2.0662311e-07 4.4877246e-16
  4.4595561e-13 9.8721902e-16 9.9999976e-01 7.4266496e-12 1.3502426e-09]
 [2.7430133e-13 1.8421916e-08 1.0000000e+00 8.0473672e-10 4.6566106e-23
  5.3524334e-15 6.8865668e-12 1.5545926e-19 2.4946656e-10 6.2741023e-21]
 [3.3679461e-09 9.9998844e-01 3.0432677e-06 4.5832249e-09 5.1557322e-07
  4.4906724e-08 3.0062827e-07 3.0446381e-06 4.4757248e-06 5.3367956e-11]
 [1.0000000e+00 1.9313025e-13 6.6222516e-09 1.6662535e-12 4.0997440e-12
  1.1336156e-11 4.1400561e-08 2.0943082e-08 3.4575329e-14 1.5246044e-09]
 [5.1374838e-10 1.4769002e-13 8.4478835e-10 2.9635687e-14 9.9992025e-01
  8.0523625e-13 4.0301367e-09 4.8266867e-07 2.1287161e-10 7.9270030e-05]]
